In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import oandapyV20
import configparser
import datetime
from datetime import datetime, timedelta

In [3]:
accountID = ""
api_key = ""

In [4]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime
# start～endまでのデータ取得
def get_period_data(start, end, minute, instrument):
    timestamp = start.timestamp()
    concats = []
    count = 5000
    while True:
        df, last_timestamp = send_api(count, timestamp, minute, instrument)
        concats.append(df)
#         print(df)
        if last_timestamp > end.timestamp() or len(df) < count:
            break
        timestamp = last_timestamp + (60 * minute)
    df = pd.concat(concats)
    if end is None:
        print(df)
        return df
    else:
        return df[df.index < end]
# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    #api_key = '取得したアクセストークンを入れてください'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1/60:
        granularity = 'S5'
    elif minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)

    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')

    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})

    return df, last_timestamp
minute = 60
start = datetime.strptime('2018-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
# end = datetime.now()
end = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
instrument = 'EUR_USD'
df = get_period_data(start, end, minute, instrument=instrument)
df = df.reset_index().drop_duplicates(subset='time',keep='first').set_index('time')
df

V20Error: {"errorMessage":"Insufficient authorization to perform request."}

In [ ]:
df_new = df.drop(columns=['complete']) #Mistake in naming the variable...:( it's not supposed to be df_open
df_new

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_new, test_size=0.20, shuffle=False)

In [ ]:
train

In [ ]:
test

In [ ]:
train.shape, test.shape

In [ ]:
window_len = 72

In [ ]:
train_lstm_in = []
for i in range(len(train)-window_len):
    temp = train[i:(i+window_len)].copy()
    for col in train:
        temp.loc[:, col] = temp[col] / temp[col].iloc[0] - 1
    train_lstm_in.append(temp)
lstm_train_out = (train['close'][window_len:].values / train['close'][:-window_len].values)-1

test_lstm_in = []
for i in range(len(test) - window_len):
    temp = test[i:(i + window_len)].copy()
    for col in test:
        temp.loc[:, col] = temp[col] / temp[col].iloc[0] - 1
    test_lstm_in.append(temp)
lstm_test_out = (test['close'][window_len:].values / test['close'][:-window_len].values)-1

In [ ]:
train_lstm_in = [np.array(train_lstm_input) for train_lstm_input in train_lstm_in]
train_lstm_in = np.array(train_lstm_in)

test_lstm_in = [np.array(test_lstm_input) for test_lstm_input in test_lstm_in]
test_lstm_in = np.array(test_lstm_in)
test_lstm_in.shape

In [ ]:
train_lstm_in.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
def build_model(inputs, output_size, neurons, activ_func="linear",
               dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()
    
    #(4944, 24, 5)
    model.add(LSTM(neurons, return_sequences=True, input_shape = (inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))

    model.add(LSTM(neurons, return_sequences=True))
    model.add(Dropout(dropout))

    model.add(LSTM(neurons, return_sequences=True))
    model.add(Dropout(dropout))

    model.add(LSTM(neurons))
    model.add(Dropout(dropout))

    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    
#     model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
#     model.add(Dropout(dropout))
#     model.add(Dense(units=output_size))
#     model.add(Activation(activ_func))
    
    model.compile(loss=loss, optimizer=optimizer)
    return model

np.random.seed(202)

yen_model = build_model(train_lstm_in, output_size=1, neurons=50)

yen_model.summary()

In [ ]:
yen_history = yen_model.fit(train_lstm_in, lstm_train_out, epochs=100, batch_size=32, verbose=1, shuffle=True)

In [ ]:
fig, ax1 = plt.subplots(1,1)

ax1.plot(yen_history.epoch, yen_history.history['loss'])
ax1.set_title('TrainingError')

if yen_model.loss == 'mae':
    ax1.set_ylabel('Mean Absolute Error (MAE)',fontsize=12)
else:
    ax1.set_ylabel('Model Loss',fontsize=12)
ax1.set_xlabel('# Epochs',fontsize=12)
plt.show()

In [ ]:
from statsmodels.tools.eval_measures import rmse
fig, ax1 = plt.subplots(1,1,figsize=(12,8))
ax1.plot(train.index[window_len:],
    train['close'][window_len:], label='Actual', color='blue', )
ax1.legend()
ax1.plot(train.index[window_len:],
    ((np.transpose(yen_model.predict(train_lstm_in))+1)*train['close'].values[:-window_len])[0],
            label='Predicted', color='red')
ax1.legend()
ax1.set_title('Prediction for Training Data({})'.format(instrument))
ax1.grid(True)

error_train = rmse(train['close'][window_len:], ((np.transpose(yen_model.predict(train_lstm_in))+1)*train['close'].values[:-window_len])[0])
print('STD of train data[close]: {}'.format(train['close'].std()))
print(f'RMSE: {error_train}')
per = round(error_train/train['close'].std()*100, 2)
print('Percentage RMSE/STD: {}%'.format(per))

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(12,8))
ax1.plot(test.index[window_len:],
    test['close'][window_len:], label='Actual', color='blue')
ax1.legend()
ax1.plot(test.index[window_len:],
    ((np.transpose(yen_model.predict(test_lstm_in))+1)*test['close'].values[:-window_len])[0],
            label='Predicted', color='red')
ax1.legend()
ax1.set_title('Prediction for Test Data({})'.format(instrument))
ax1.grid(True)

error_test = rmse(test['close'][window_len:], ((np.transpose(yen_model.predict(test_lstm_in))+1)*test['close'].values[:-window_len])[0])
print('STD of test data[close]: {}'.format(test['close'].std()))
print(f'RMSE: {error_test}')
per = round(error_test/test['close'].std()*100, 2)
print('Percentage RMSE/STD: {}%'.format(per))

In [ ]:
# train['close'].std()

In [ ]:
# df_new.columns[3]

In [ ]:
# yen_model.save("LSTM_FX_Model")

In [ ]:
# from keras.models import load_model
# model = load_model("LSTM_FX_Model")
# model

In [ ]:
# model.summary()